Zunächst müssen die notwendigen Pakete geladen werden.

In [ ]:
using ControlSystems
using LinearAlgebra
using Plots
using Interact
using Printf

# Aufgabe 1

Aufstellen der Zustandsraumdarstellung aus der Aufgabe.

In [ ]:
A, b, c, d = 1, 5, 2, 0

In [ ]:
sys = ss(A, b, c, d)

## Aufgabe 1.1

Die Sprung- und Stoßantwort des Systems werden 5 Sekunden lang simuliert.

In [ ]:
t_sim = 5
plot(step(sys, t_sim), label="Sprungantwort")
plot!(impulse(sys, t_sim), label="Stoßantwort")

Das System ist nicht bibo-stabil, da die Sprungantwort nicht amplitudenbeschränkt ist.

## Aufgabe 1.2

Der Zustandsregler hat als Parameter die Rückführung $R$ und das Vorfilter $V$, die eingestellt werden können.

In [ ]:
@manipulate for R = 0.1:0.01:2, V = 0.05:0.001:1
    # Resultierender Eigenwert
    λ = 1 - 5R

    # Zustandsraumdarstellung des geschlossenen Regelkreises mit Zustandsregler
    sys_cl = ss(A - b * R, V * b, c, d)

    # Simuliere Gesamtsystem
    res = step(sys_cl, t_sim)
    # Stellgröße u
    u = V * res.u' - R * res.x'
    
    p = plot(res, label="y(t)", ylim=(-0.5, 2.5))
    plot!(res.t, u, label="u(t)")
    plot!(t -> 1, label="σ(t)")
    # Ausgabe von Eigenwert und Plot
    hbox(@sprintf("λ = %.2f", λ), hskip(5em), p)
end

Nun werden Rückführung und Vorfilter so bestimmt, dass das resultierende Gesamtsystem den gewünschten Eigenwert $\lambda$ hat und stationär genau ist.

In [ ]:
@manipulate for λ = -5:0.01:-0.1
    # Rückführung für gewünschten Eigenwert
    R = (1 - λ) / 5
    # Vorfilter für stationäre Genauigkeit
    V = -λ / 10

    # Zustandsraumdarstellung des geschlossenen Regelkreises mit Zustandsregler
    sys_cl = ss(A - b * R, V * b, c, d)

    # Simuliere Gesamtsystem
    res = step(sys_cl, t_sim)
    # Stellgröße u
    u = V * res.u' - R * res.x'
    
    p = plot(res, label="y(t)", ylim=(-0.5, 2.5))
    plot!(res.t, u, label="u(t)")
    plot!(t -> 1, label="σ(t)")
    # Zeige Parameter und Plot
    hbox(
        vbox(
            @sprintf("R = %.2f", R), vskip(0),
            @sprintf("V = %.2f", V)
        ), hskip(5em), p
    )
end

## Aufgabe 1.3

Nun wird ein P-Regler mit Verstärkung $K$ eingesetzt.

In [ ]:
@manipulate for K = 0.05:0.01:2
    # Resultierender Eigenwert
    λ = 1 - 10K

    # Zustandsraumdarstellung des geschlossenen Regelkreises mit P-Regler
    sys_cl = ss(A - K * b * c, K * b, c, d)
    
    # Simuliere Gesamtsystem
    res = step(sys_cl, t_sim)
    # Stellgröße u
    u = K * (res.u' - res.y')
    
    p = plot(res, label="y(t)", ylim=(-0.5, 2.5))
    plot!(res.t, u, label="u(t)")
    plot!(t -> 1, label="σ(t)")
    # Ausgabe von Eigenwert und Plot
    hbox(@sprintf("λ = %.2f", λ), hskip(5em), p)
end

# Aufgabe 2

Zur Stabilitätsüberprüfung mit Hilfe eines Computers können die Eigenwerte der Systemmatrix einfach berechnet werden.

In [ ]:
M = [ 0  1  0
      0  0  1
     -7 -3 -2 ]

In [ ]:
v = eigvals(M)

Überprufe ob alle Eigenwerte negativen Realteil haben:

In [ ]:
all(real(v) .< 0) # der Punkt . bewirkt die elementweise Auswertung des Operators <